### Regression in R

Running regressions with **R** is quite easy. Later in the course we'll get into some more complex regression commands, but for now we'll stick with simple linear regression using the `lm()` command. 

First, let's take a step back and see how mpg and price are correlated in the data. We can obtain correlations of two variables using the `cor()` function. 

In [1]:
# First let's load our data and take a look
carsdata <- read.csv("autos.csv")
head(carsdata)

,X,make,price,mpg,rep78,headroom,trunk,weight,length,turn,displacement,gear_ratio,foreign
,<int>,<chr>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>
1,1,AMC Concord,4099,22,3,2.5,11,2930,186,40,121,3.58,0
2,2,AMC Pacer,4749,17,3,3.0,11,3350,173,40,258,2.53,0
3,3,AMC Spirit,3799,22,NA,3.0,12,2640,168,35,121,3.08,0
4,4,Buick Century,4816,20,3,4.5,16,3250,196,40,196,2.93,0
5,5,Buick Electra,7827,15,4,4.0,20,4080,222,43,350,2.41,0
6,6,Buick LeSabre,5788,18,3,4.0,21,3670,218,43,231,2.73,0


In [2]:
# Now we can look at the correlation between mpg and price
mpg_weight_cor <- cor(carsdata$mpg, carsdata$price)
mpg_weight_cor

[1] -0.4685967

What if we used a simple OLS regression instead? With regression functions like `lm()`, the first argument is the regression formula. Here we specify `price ~ mpg`, so we're telling R to describe *price* as a function of *mpg*. In practice, this is estimating the equation

$$ price_i = \beta_0 + \beta_1 mpg_i + \epsilon_i $$ 

where *price* is the dependent variable and *mpg* the independent variable/covariate. $\beta_0$ and $\beta_1$ are our intercept and slope coefficients, respectively - we'll spend a bunch of time learning about and interpreting these coefficients throughout the class. $\epsilon_i$ is an error term.

The second argument is the name of the dataset.

In [3]:
price_mpg_regression <- lm(price ~ mpg, data = carsdata)
price_mpg_regression


Call:
lm(formula = price ~ mpg, data = carsdata)

Coefficients:
(Intercept)          mpg  
    11253.1       -238.9  


Notice how we stored the model as an object. This is very important, since the lm object contains a lot of useful information about our model that we will want to access later. Also, notice that simply calling `price_mpg_regression` only output the regression equation and the coefficients. In order to get a better look at your output, including the estimate, standard error, t-value, p-value, R-squared and more, use the `summary()` command. The code below produces a summary of our model:

In [4]:
summary(price_mpg_regression)


Call:
lm(formula = price ~ mpg, data = carsdata)

Residuals:
    Min      1Q  Median      3Q     Max 
-3184.2 -1886.9  -959.8  1359.7  9669.7 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 11253.06    1170.81   9.611 1.53e-14 ***
mpg          -238.89      53.08  -4.501 2.55e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2624 on 72 degrees of freedom
Multiple R-squared:  0.2196,	Adjusted R-squared:  0.2087 
F-statistic: 20.26 on 1 and 72 DF,  p-value: 2.546e-05


We can also obtain the fitted values and residuals by calling them with `name$fitted.values` and `name$residuals`, respectively. Use the below code to look at the first few fitted values and the average value of the regression residuals.

In [10]:
head(price_mpg_regression$fitted.values)
mean(price_mpg_regression$residuals)

1        2        3        4        5        6 
5997.385 7191.857 5997.385 6475.174 7669.645 6952.962

[1] -6.76936e-14

We can also use indexing (discussed in Coding Bootcamp part 3 in greated detail) to access the esimated parameters stored in the lm object. There are a few ways to do this, the first is through calling `name$coefficient[i]`, where i = 1 will give you the intercept, i = 2 will give you the first explanatory variable and so forth. The code below calls the Intercept and $\beta_1$ from the regression:

In [15]:
price_mpg_regression$coefficient[1]
price_mpg_regression$coefficient[2]

(Intercept) 
   11253.06

mpg 
-238.8943

Another way to access the coefficients is through the lm summary, which stores coefficient, standard errors, t-values and p-values in a matrix. By properly indexing the stored matrix, we have access to not only the coefficients but also the associated standard error and test statistics. To access the stored information, call them by `summary(name)$coef[i,j]`. The intercept and explanatory variables are indexed by rows (i), and the estimate, standard error and test statistics are indexed by columns (j). We can access either the entire row, or each cell individually. 

For example, the following code calls the entire row of stored information for the dependent variable (mpg):

In [19]:
summary(price_mpg_regression)$coef[2,]

Estimate    Std. Error       t value      Pr(>|t|) 
-2.388943e+02  5.307669e+01 -4.500928e+00  2.546131e-05

If instead we want to call each stored object individually, we can call on specific columns. j = 1 accesses the _coefficient estimate_, j = 2 accesses the _standard error_, j = 3 accesses the _t-value_ and j = 4 accesses the _p-value_. The code below calls the point estimate and standard error for the model's explanatory variable (mpg).

In [22]:
print(paste0('The point estimate is ', summary(price_mpg_regression)$coef[2,1]))
print(paste0('The point standard error is ', summary(price_mpg_regression)$coef[2,2]))

[1] "The point estimate is -238.89434563313"
[1] "The point standard error is 53.0766871613496"


Finally, we can use the lm object for prediction. As described above, OLS predictions of $\hat{y}$ for observations in the sample are stored in the lm object to be called by `name$fitted.value`. But we can also predict values of y for observations _not_ in the original estimation sample. As long as we have data on the explanatory variables in the model, we can use our lm object to predict a value for the outcome using the `predict()` command from the lm library. 

The following code generates example data on miles per gallon and uses the `predict()` function to obtain predicted prices using the OLS model estimated on our original data:

In [23]:
# this line creates a dataframe of 10 random numbers between the min and max of the original data mpg
# you do not need to know how to do this, but it is useful! 
new_data <- data.frame(mpg = runif(10, min = min(carsdata$mpg), max = max(carsdata$mpg))) 

# now we can predict the price for our new data on mpg
price_predictions <- predict(price_mpg_regression, new_data)
price_predictions

1        2        3        4        5        6        7        8 
2783.742 2700.981 2483.248 5278.579 6320.824 5022.304 2559.685 8122.069 
       9       10 
5701.771 5874.652

Note the syntax of the predict function. It is `predict(lm object, data frame)`: the first argument is the stored lm object of the model you want to use for prediction and the second object is a _dataframe_ of new values.

The following code will produce errors since it is not stored as a data.frame object.

In [24]:
mpg <- runif(10, min = min(carsdata$mpg), max = max(carsdata$mpg))
predict(price_mpg_regression, mpg)

ERROR: Error in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$xlevels): 'data' must be a data.frame, environment, or list


In [29]:
mpg <- as.data.frame(mpg) # a simple fix!
predict(price_mpg_regression, mpg)

1        2        3        4        5        6        7        8 
4818.212 7346.132 1478.312 7968.097 7298.880 5102.983 5343.331 8128.346 
       9       10 
2377.313 7581.063